# Tutorial 0: Basic examples

Synthetic tabular data is a type of data that is artificially generated, rather than collected from real-world sources. It is designed to mimic the structure and characteristics of real-world data, and is often used in a variety of applications, including machine learning, data analysis, and testing.

Synthetic tabular data is usually created by combining different elements from real-world data sets, and then applying various transformations and randomization techniques to create new, artificial data. This allows researchers to study and test algorithms and models on data that is similar to what they would encounter in the real world, but without the need to collect and use sensitive or confidential real-world data.

`synthcity` supports generating tabular data for various modalities. In this tutorial we will cover the general-purpose models.

In [ ]:
!pip install synthcity
!pip uninstall -y torchaudio torchdata

In [ ]:
# stdlib
import sys
import warnings

warnings.filterwarnings("ignore")

# third party
from sklearn.datasets import load_diabetes

# synthcity absolute
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

log.add(sink=sys.stderr, level="INFO")

X, y = load_diabetes(return_X_y=True, as_frame=True)
X["target"] = y

X

In [ ]:
# Preprocessing data with OneHotEncoder or StandardScaler is not needed or recommended. Synthcity handles feature encoding and standardization internally.
loader = GenericDataLoader(
    X,
    target_column="target",
    sensitive_columns=["sex"],
)

## List the available generative models

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

Plugins().list()

## Load and train a generative model

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

syn_model = Plugins().get("marginal_distributions")

syn_model.fit(loader)

## Generate new data using the model

In [ ]:
syn_model.generate(count=10).dataframe()

## Generate new data using conditionals

We will condition the generated data using the target data(y).

In [ ]:
# synthcity absolute
from synthcity.plugins import Plugins

syn_model = Plugins().get("adsgan")

syn_model.fit(loader, cond=y.to_frame())

In [ ]:
import numpy as np

count = 10
syn_model.generate(
    count=count, cond=np.ones(count)
).dataframe()  # generate only patients with the outcome = 1

## Serialization

In [ ]:
# synthcity absolute
from synthcity.utils.serialization import load, load_from_file, save, save_to_file

buff = save(syn_model)

type(buff)

In [ ]:
reloaded = load(buff)

reloaded.name()

## Plot real-synthetic distributions

In [ ]:
# third party
import matplotlib.pyplot as plt

syn_model.plot(plt, loader)

plt.show()

## Benchmarking metrics

| **Metric**                                         | **Description**                                                                                                            |
|----------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------|
| sanity.data\_mismatch.score                        | Data types mismatch between the real//synthetic features                                                                   |
| sanity.common\_rows\_proportion.score              | Real data copy-paste in the synthetic data                                                                                 |
| sanity.nearest\_syn\_neighbor\_distance.mean       | Computes the \textless{}reduction\textgreater{}(distance) from the real data to the closest neighbor in the synthetic data |
| sanity.close\_values\_probability.score            | the probability of close values between the real and synthetic data.                                                       |
| sanity.distant\_values\_probability.score          | the probability of distant values between the real and synthetic data.                                                     |
| stats.jensenshannon\_dist.marginal                 | the average Jensen-Shannon distance                                                                                        |
| stats.chi\_squared\_test.marginal                  | the one-way chi-square test.                                                                                               |
| stats.feature\_corr.joint                          | the correlation/strength-of-association of features in data-set with both categorical and continuous features              |
| stats.inv\_kl\_divergence.marginal                 | the average inverse of the Kullback–Leibler Divergence metric.                                                             |
| stats.ks\_test.marginal                            | the Kolmogorov-Smirnov test for goodness of fit.                                                                           |
| stats.max\_mean\_discrepancy.joint                 | Empirical maximum mean discrepancy. The lower the result the more evidence that distributions are the same.                |
| stats.prdc.precision                               | precision between the two manifolds                                                                                        |
| stats.prdc.recall                                  | recall between the two manifolds                                                                                           |
| stats.prdc.density                                 | density between the two manifolds                                                                                          |
| stats.prdc.coverage                                | coverage between the two manifolds                                                                                         |
| stats.alpha\_precision.delta\_precision\_alpha\_OC | Delta precision                                                                                                            |
| stats.alpha\_precision.delta\_coverage\_beta\_OC   | Delta coverage                                                                                                             |
| stats.alpha\_precision.authenticity\_OC            | Authetnticity                                                                                                              |
| performance.linear\_model.gt.aucroc              | Train on real, test on the test real data using LogisticRegression: AUCROC                                                             |
| performance.linear\_model.syn\_id.aucroc         | Train on synthetic, test on the train real data using LogisticRegression: AUCROC                                                       |
| performance.linear\_model.syn\_ood.aucroc        | Train on synthetic, test on the test real data using LogisticRegression: AUCROC                                                        |
| performance.mlp.gt.aucroc                        | Train on real, test on the test real data using NN: AUCROC                                                                |
| performance.mlp.syn\_id.aucroc                    | Train on synthetic, test on the train real data using NN: AUCROC                                                          |
| performance.mlp.syn\_ood.aucroc                   | Train on synthetic, test on the test real data using NN: AUCROC                                                           |
| performance.xgb.gt.aucroc                         | Train on real, test on the test real data using XGB: AUCROC                                                               |
| performance.xgb.syn\_id.aucroc                    | Train on synthetic, test on the train real data using XGB: AUCROC                                                         |
| performance.xgb.syn\_ood.aucroc                   | Train on synthetic, test on the test real data using XGB: AUCROC                                                          |
| performance.feat\_rank\_distance.corr              | Correlation for the rank distances between the feature importance on real and synthetic data                               |
| performance.feat\_rank\_distance.pvalue            | p-vale for the rank distances between the feature importance on real and synthetic data                                    |
| detection.detection\_xgb.mean                      | The average AUCROC score for detecting synthetic data using an XGBoost.                                                    |
| detection.detection\_mlp.mean                      | The average AUCROC score for detecting synthetic data using a NN.                                                          |
| detection.detection\_gmm.mean                      | The average AUCROC score for detecting synthetic data using a GMM.                                                         |
| privacy.delta-presence.score                       | the maximum re-identification probability on the real dataset from the synthetic dataset.                                  |
| privacy.k-anonymization.gt                         | the k-anon for the real data                                                                                               |
| privacy.k-anonymization.syn                        | the k-anon for the synthetic data                                                                                          |
| privacy.k-map.score                                | the minimum value k that satisfies the k-map rule.                                                                         |
| privacy.distinct l-diversity.gt                    | the l-diversity for the real data                                                                                          |
| privacy.distinct l-diversity.syn                   | the l-diversity for the synthetic data                                                                                     |
| privacy.identifiability\_score.score               | the re-identification score on the real dataset from the synthetic dataset.                                                |

## Benchmark the quality of plugins

In [ ]:
# synthcity absolute
from synthcity.benchmark import Benchmarks

score = Benchmarks.evaluate(
    [
        ("marginal_distributions", "marginal_distributions", {}),
        ("dummy_sampler", "dummy_sampler", {}),
    ],
    loader,
    synthetic_size=1000,
    repeats=2,
)

In [ ]:
Benchmarks.print(score)

## Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star [Synthcity](https://github.com/vanderschaarlab/synthcity) on GitHub

- The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.


### Checkout other projects from vanderschaarlab
- [HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
- [AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
